# Numerical Integration


solve_ivp is the 'standard' numerical integration interface in scientific python<br>
It contains a number of integrators, many of which are available in other languages
(R, Matlab etc) - often using exactly the same backend implementation

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

In [ ]:
# Let's start with a simple SIR model as our toy integration problem

def sir_step(t, values, rate_dict):
    s, i, r = values
    
    pop = sum(values)
    iprop = i / pop
    
    s_i = s * rate_dict["s_i"] * iprop
    i_r = i * rate_dict["i_r"]
    
    return np.array([
           -s_i, s_i - i_r, i_r
    ])
    

In [ ]:
init_values = np.array((100.0,1.0,0.0))

times = np.arange(0,200)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

sir_rates = {
    "s_i": 0.9,
    "i_r": 0.1
}

# RK45 is the default method
# This is an explicit adaptive multistep method
method = "RK45"

# solve_ivp requires the following arguments:
# ode_function, (start,end), initial_values, evaluation_times, method, <optional arguments>

res = solve_ivp(sir_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sir_rates])

In [ ]:
# Let's be honest - most applications will never actually check this, but let's have a look
# in detail...

res

In [ ]:
pd.DataFrame(res["y"].T,columns=["S","I","R"],index=times).plot()

In [ ]:
# Just for comparison, let's write out the simplest integrator we can - Euler's method 

def solve_euler(func, times, init_values, args=None):
    h = times[1] - times[0]
    values = init_values
    y_res = [init_values]
    for i, t in enumerate(times[:-1]):
        
        # This is it - the core loop
        # evaluation the function at t, then add our deltas (times the timestep (h))
        y = func(t, values, args)
        values = values + y * h
        
        # Gather these up
        y_res.append(values)
    return np.stack(y_res)

In [ ]:
times = np.arange(0,200)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

sir_rates = {
    "s_i": 0.9,
    "i_r": 0.1
}

eres = solve_euler(sir_step, times, init_values, sir_rates)

pd.DataFrame(eres,columns=["S","I","R"],index=times).plot()

In [ ]:
# Without a doubt, the most widely used multistep method, Runge-Kutta 4

def solve_rk4(ode_func, times, init_values, args=None):
    
    h = times[1] - times[0]
    values = init_values
    y_res = [init_values]
    
    # Perform Runge-Kutta 4 integration
    for time_idx, time in enumerate(times[:-1]):
        
        k1 = h * ode_func(time, values, args)
        k2 = h * ode_func(time + h / 2, values + k1 / 2, args)
        k3 = h * ode_func(time + h / 2, values + k2 / 2, args)
        k4 = h * ode_func(time + h, values + k3, args)
        
        values = values + (1 / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
        
        y_res.append(values)

    return np.stack(y_res)

In [ ]:
times = np.arange(0,200)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

sir_rates = {
    "s_i": 0.9,
    "i_r": 0.1
}

rres = solve_rk4(sir_step, times, init_values, sir_rates)

pd.DataFrame(rres,columns=["S","I","R"],index=times).plot()

In [ ]:
# Let's try a slightly more complex (if not necessarily realistic) function
# SIRS with optional reinfection from the recovered compartment

# Functions with feedback are considerably more demanding regarding stability

def sirs_step(t, values, rate_dict):
    s, i, r = values
    
    pop = sum(values)
    iprop = i / pop
    
    s_i = s * rate_dict["s_i"] * iprop
    i_r = i * rate_dict["i_r"]
    r_s = r * rate_dict["r_s"]
    r_i = r * rate_dict["r_i"] * iprop
    
    return np.array([
           -s_i + r_s, s_i + r_i - i_r, i_r - r_s - r_i
    ])
    

In [ ]:
times = np.arange(0,2000)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

sirs_rates = {
    "s_i": 0.5,
    "i_r": 0.3,
    "r_s": 0.003,
    "r_i": 0.05,
}

method = "RK45"
res = solve_ivp(sirs_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sirs_rates])

outputs = res["y"].T

pd.DataFrame(outputs,columns=["S","I","R"]).plot()

In [ ]:
# Now try some slightly more 'challenging' values - nothing extreme

times = np.arange(0,2000)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

sirs_rates = {
    "s_i": 0.94,
    "i_r": 0.6,
    "r_s": 0.001,
    "r_i": 0.1,
}

In [ ]:
# We'll start with RK45, but try a few different methods here

# BDF, Radau

method = "RK45"
res = solve_ivp(sirs_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sirs_rates])

outputs = res["y"].T

res

In [ ]:
pd.DataFrame(outputs,columns=["S","I","R"]).plot()

In [ ]:
outputs = solve_euler(sirs_step, times, init_values, args=sirs_rates)
pd.DataFrame(outputs,columns=["S","I","R"]).plot()

In [ ]:
outputs = solve_rk4(sirs_step, times, init_values, args=sirs_rates)
pd.DataFrame(outputs,columns=["S","I","R"]).plot()

In [ ]:
# Let's go back to SIR, but add an importation step

def sir_import_step(t, values, rate_dict):
    s, i, r = values
    
    pop = sum(values)
    iprop = i / pop
    
    s_i = s * rate_dict["s_i"] * iprop
    i_r = i * rate_dict["i_r"]
    
    if t >= 100 and t < 101:
        imports = 20.0
    else:
        imports = 0.0
        
    
    return np.array([
           -s_i, s_i - i_r + imports, i_r
    ])
    

In [ ]:
times = np.arange(0,200)

n_substeps = 1

times = np.linspace(0.0,times[-1],int(len(times) * n_substeps))

# We'll start with these values...
sir_rates = {
    "s_i": 0.4,
    "i_r": 0.14 # 7 day recovery
}

method = "RK45"
res = solve_ivp(sir_import_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sir_rates])

outputs = res["y"].T

pd.DataFrame(outputs,columns=["S","I","R"],index=times).plot()

In [ ]:
res

In [ ]:
# A slight change

sir_rates = {
    "s_i": 0.4,
    "i_r": 0.12 # 8 day recovery
}


method = "RK45"
res = solve_ivp(sir_import_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sir_rates])

outputs = res["y"].T

pd.DataFrame(outputs,columns=["S","I","R"],index=times).plot()

In [ ]:
res

In [ ]:
# Different integrators behave differently - of course they do, they all have different
# adaptation strategies...

sir_rates = {
    "s_i": 0.4,
    "i_r": 0.14 # 7 day recovery
}


method = "BDF"
res = solve_ivp(sir_import_step, (times[0],times[-1]), init_values, t_eval=times,method=method,args=[sir_rates])

outputs = res["y"].T

pd.DataFrame(outputs,columns=["S","I","R"],index=times).plot()

In [ ]:
# A slight change

sir_rates = {
    "s_i": 0.4,
    "i_r": 0.12 # 8 day recovery
}

# max_step - this is what really affects integration intervals (not the 'evaluation times')
# ... but at a cost

method = "RK45"
res = solve_ivp(sir_import_step,
                (times[0],times[-1]),
                init_values,
                t_eval=times,
                method=method,
                args=[sir_rates],
                max_step=1.0) 


outputs = res["y"].T

pd.DataFrame(outputs,columns=["S","I","R"],index=times).plot()

In [ ]:
res

In [ ]:
# Maybe we were better off with RK4 in the first place? (...maybe?)
# In terms of granularity, RK4 has a "max_step" of h/2

rres = solve_rk4(sir_import_step, times, init_values, sir_rates)

pd.DataFrame(rres,columns=["S","I","R"],index=times).plot()

# Some other properties

In [ ]:
TAU = 2.0 * np.pi

In [ ]:
def hooke_ode(t, values, k):
    x, v = values
    a = -k * x
    return np.array((v, a))

In [ ]:
init_hooke = np.array((1.0, 0.0))

N_CYCLES = 2
STEP_DIVISOR = 32
END = TAU * N_CYCLES

times = np.linspace(0.0,END,N_CYCLES * STEP_DIVISOR)

method = "RK45"
%time res = solve_ivp(hooke_ode, [0,END], init_hooke, t_eval=times,method=method,args=[1.0])

#pd.Series(data=res["y"][0],index=times).plot()
pd.DataFrame(data=res["y"][0].T,index=times).plot()

In [ ]:
# 
init_hooke = np.array((1.0, 0.0))

N_CYCLES = 1000
STEP_DIVISOR = 16
END = TAU * N_CYCLES

times = np.linspace(0.0,END,N_CYCLES * STEP_DIVISOR)

method = "RK45"
%time res = solve_ivp(hooke_ode, [0,END], init_hooke, t_eval=times,method=method,args=[1.0e1])

#pd.Series(data=res["y"][0],index=times).plot()
pd.DataFrame(data=res["y"][0].T,index=times).plot()

In [ ]:
res

# Reframe this as a symplectic integrator

In [ ]:
def verlet_hooke(values,h, t, k):
    xn, xn1 = values
    a = -k * xn
    
    x_new = 2.0 * xn - xn1 + a * (h**2.0)
    return np.array((x_new, xn))

def solve_verlet(times, init_values, h, k):
    values = init_values
    y_res = [init_values]
    for t in times[::-1]:
        values = verlet_hooke(values, h, t, k)
        y_res.append(values)
    return np.stack(y_res).T

In [ ]:
init_hooke = np.array((1.0, 1.0))

N_CYCLES = 1000
STEP_DIVISOR = 16
END = TAU * N_CYCLES

times = np.linspace(0.0,END,N_CYCLES * STEP_DIVISOR)

h = END / ((N_CYCLES * STEP_DIVISOR)-1)

vres = solve_verlet(times, init_hooke, h, 1.0e1)
pd.Series(data=vres[0]).plot()

In [ ]:
# Now finally, let's take a look at some stiff non-linear problems...

In [ ]:
def hooke_nl_ode(t, values, k):
    x, v = values
    a = -k * (x + ((10.0 * x)**3))
    return np.array((v, a))

In [ ]:
init_hooke = np.array((1.0, 0.0))

N_CYCLES = 4
STEP_DIVISOR = 256
END = TAU * N_CYCLES

times = np.linspace(0.0,END,N_CYCLES * STEP_DIVISOR)

# Try this with different methods:
# RK45, BDF, Radau, LSODA
method = "RK45"
%time res = solve_ivp(hooke_nl_ode, [0,END], init_hooke, t_eval=times,method=method,args=[1.0e1])

#pd.Series(data=res["y"][0],index=times).plot()
pd.DataFrame(data=res["y"][0].T,index=times).plot()